In [1]:
import os
import numpy as np
from matplotlib.pyplot import *
import knmi
import scipy
import netCDF4
import waterbase
import pandas as pd
import cPickle as pickle
from datetime import datetime
import aeolis

root = r'D:\Documents\GIT_checkouts\aeolis-models\sandmotor'

t0 = datetime(2011,8,1)
t1 = datetime(2012,8,1)

key = '20110803'

# Model setup

In [120]:
# read wind time series
meteo, legend = knmi.read_uurgeg(os.path.join(root, 'uurgeg_330_2011-2020.txt'))
ix = (meteo['DD'] > 360.) | (meteo['FH'] >= 900)
meteo = meteo.ix[~ix]


In [3]:
# write wind time series
alpha = 48.
m = meteo[t0:t1].filter(['FH','DD'])
t = np.asarray([[x.total_seconds() for x in m.index - m.index[0]]]).T

m['FH'] = m['FH'].divide(10.)
m['DD'] = m['DD'].subtract(alpha)

np.savetxt(os.path.join(root, 'wind.txt'), np.concatenate((t, m.as_matrix()), axis=1))

In [129]:
# write wind time series (10yr)
alpha = 48.
m = meteo[t0:t1].filter(['FH','DD'])
t = np.asarray([[x.total_seconds() for x in m.index - m.index[0]]]).T
   
m['FH'] = m['FH'].divide(10.)
m['DD'] = m['DD'].subtract(alpha)

#multiply the years
yr = 5
t1yr = np.concatenate((t, m.as_matrix()), axis=1)
tall = np.tile(t1yr,(yr,1))
tfac = np.repeat(range(0,yr),t1yr[:,0].shape,axis=0)
tall[:,0] = tall[:,0]+tfac*t1yr[-1,0] 

np.savetxt(os.path.join(root, 'wind_5yr.txt'), tall)

In [10]:
# read and write water levels (10yr)
wb = waterbase.Waterbase()
wl = wb.get_data('1', 'HOEKVHLD', t0, t1)
t = np.asarray([[(x - wl['datetime'][0]).total_seconds() for x in wl['datetime']]]).T
wl1yr = np.concatenate((t, wl['value'].reshape((-1,1))/100.), axis=1)

yr = 10
wlall = np.tile(wl1yr,(yr,1))
wlfac = np.repeat(range(0,yr),wl1yr[:,0].shape,axis=0)
wlall[:,0] = wlall[:,0]+wlfac*wl1yr[-1,0] 


np.savetxt(os.path.join(root, 'tide_10yr.txt'), wlall)

In [11]:
# read and write water levels 
wb = waterbase.Waterbase()
wl = wb.get_data('1', 'HOEKVHLD', t0, t1)
t = np.asarray([[(x - wl['datetime'][0]).total_seconds() for x in wl['datetime']]]).T

wl = np.concatenate((t, wl['value'].reshape((-1,1))/100.), axis=1)

np.savetxt(os.path.join(root, 'tide.txt'), wl)

In [12]:
# read and write wave heights
Hs = wb.get_data('22', 'Euro platform', t0, t1)
t = np.asarray([[(x - Hs['datetime'][0]).total_seconds() for x in Hs['datetime']]]).T
Hs = np.concatenate((t, Hs['value'].reshape((-1,1))/100.), axis=1)

np.savetxt(os.path.join(root, 'waves.txt'), Hs)

In [10]:
# read and write wave heights (10 yrs)
wb = waterbase.Waterbase()
Hs = wb.get_data('22', 'Euro platform', t0, t1)
t = np.asarray([[(x - Hs['datetime'][0]).total_seconds() for x in Hs['datetime']]]).T
Hs1yr = np.concatenate((t, Hs['value'].reshape((-1,1))/100.), axis=1)

yr = 5
Hsall = np.tile(Hs1yr,(yr,1))
Hsfac = np.repeat(range(0,yr),Hs1yr[:,0].shape,axis=0)
Hsall[:,0] = Hsall[:,0]+Hsfac*Hs1yr[-1,0] 


np.savetxt(os.path.join(root, 'waves_5yr.txt'), Hsall)

In [22]:
aeolis.model.AeoLiSRunner(configfile='joepie').run()

**********************************************************
 
         d8888                   888      d8b  .d8888b.   
        d88888                   888      Y8P d88P  Y88b  
       d88P888                   888          Y88b.       
      d88P 888  .d88b.   .d88b.  888      888  "Y888b.    
     d88P  888 d8P  Y8b d88""88b 888      888     "Y88b.  
    d88P   888 88888888 888  888 888      888       "888  
   d8888888888 Y8b.     Y88..88P 888      888 Y88b  d88P  
  d88P     888  "Y8888   "Y88P"  88888888 888  "Y8888P"   
 
**********************************************************
PARAMETER SETTINGS
**********************************************************
                A = 100.000000
               Cb = 1.500000
                F = 0.000100
                T = 1.000000
         bed_file = bed.txt
     bedcomp_file = <novalue>
        bedupdate = T
               bi = 1.000000
               dt = 60.000000
              eps = 0.001000
      evaporation = T
           facDOD = 

# Plot results 

In [5]:
def create_plot(par, data):
    
    # remove inf's
    ix = np.isinf(data)
    data[ix] = np.nan
    
    # remove extra dimensions
    if data.ndim == 4:
        data = data[:,:,0,:] # pick top layer
    if data.ndim == 3:
        data = data.sum(axis=-1) # sum over fractions
    
    # determine data range
    mn = np.nanmin(data[1:-1,1:-1])
    mx = np.nanmax(data[1:-1,1:-1])
    mx = np.maximum(np.abs(mn), np.abs(mx))

    if mn > -1e-10:
        mn = 0.
        cmap = 'Reds'
    else:
        mn = -mx
        cmap = 'bwr'

    # create plot
    fig, axs = subplots(figsize=(10,4))
    p = axs.pcolormesh(y, x, data, cmap=cmap, vmin=mn, vmax=mx)
    axs.set_aspect('equal')
    axs.set_title(par)
    fig.colorbar(p)

    # save plot
    fig.savefig(os.path.join(root, 'aeolis_5year_sorting_noHydro', '%s.png' % par))

    
n = 10 # number of time steps
with netCDF4.Dataset(os.path.join(root, 'aeolis_5year_sorting_noHydro.nc'), 'r') as ds:
    t = ds.variables['time'][:] / 3600.
    x = ds.variables['x'][:,:]
    y = ds.variables['y'][:,:]
    f = ds.variables['fractions'][:]
    
    nx = x.shape[1]
    ny = y.shape[0]
    
    for t in np.linspace(0, len(t)-1, n):
        
        t = int(np.round(t))
    
        # bathy difference
        data = ds.variables['zb'][...]
        create_plot('dz_t=%d' % t, data[t,:,:] - data[0,:,:])

        # mean grain size
        data = ds.variables['mass'][...]
        gs = ds.groups['settings'].getncattr('grain_size') * 1e6
        #create_plot('d50_t=%d' % t,
        #            np.asarray([np.average(gs, weights=ds.variables['mass'][t,j,i,0,:])
        #                        for j in range(ny) for i in range(nx)]).reshape((ny, nx)))
        
        # other variables
        for par in ['Ct', 'Cu', 'pickup', 'zb', 'zs']:
            create_plot('%s_t=%d' % (par, t), ds.variables[par][t,...])

C:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:227: RuntimeWarning: All-NaN axis encountered
  warnings.warn("All-NaN axis encountered", RuntimeWarning)
C:\Anaconda\lib\site-packages\numpy\lib\nanfunctions.py:326: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)


In [36]:
print t

10


[  731.  1462.  2193.  2924.  3655.  4386.  5117.  5848.  6579.  7310.
  8041.]
